In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI
from dotenv import load_dotenv
import os
import random
from storysim import StorySimulator
import pandas as pd
from together import Together

/homes/ngetach/miniconda/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')
os.environ['TOGETHER_API_KEY'] = os.getenv('TOGETHER_KEY')


In [5]:
# This example is the new way to use the OpenAI lib for python


def prompt_gpt(prompt, model):
    if 'gpt' in model:
        client = OpenAI()
    else:
        client = Together()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    
    return response.choices[0].message.content

### Experiment Setup

In [6]:
def mislead_experiment(actors, locs, g, mislead, length):
    poi = random.sample(actors, 2)
    loc = random.sample(locs, 1)
    second_loc = random.choice(g[loc[0]])
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc, "path_type": "same"}
    event_dict[11] = {"name":"move", "actor":poi[-1], "location": second_loc}
    event_dict[12] = {"name": "exclusive_random", "actors": poi, "stop": 12 + mislead}
    event_dict[12 + mislead] = {"name": "mislead", "actors": poi}
    event_dict[12 + mislead+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, second_loc, experiment_info

def spaced_mislead_experiment(actors, locs, g, mislead, length):
    event_dict = {}
    poi = random.sample(actors, 2)
    loc = random.sample(locs, 1)
    label = random.choice([l for l in g[loc[-1]] if l != loc[-1]])
    event_dict[15] = {"name": "cross_paths","actors": poi, "location": loc}
    event_dict[16] = {"name": "exclusive_random", "actors": poi, "stop": 17}
    event_dict[17] = {"name":"move", "actor":poi[-1],"location":label}
    event_dict[18] = {"name": "exclusive_random", "actors": poi, "stop": 18+mislead}
    event_dict[18+mislead] = {"name": "mislead", "actors": poi}
    event_dict[18 + mislead +1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, label, experiment_info

def number_of_moves_experiment(actors, locs, g, length):
    poi = random.sample(actors, 2)
    loc = random.sample(locs,1)
    num_moves = 0
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc, "path_type":"same"}
    prev = loc[0]
    movement = []
    for i in range(1,num_moves+1):
        new_loc = random.choice([l for l in g[prev] if l not in loc])
        movement.append(new_loc)
        event_dict[10+i] = {"name":"move", "actor":poi[-1], "location": new_loc}
        prev = new_loc
    #event_dict[10+num_moves+1] = {"name": "mislead", "actors": poi}
    label =  movement[0]
    event_dict[10+num_moves+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, label, experiment_info

def second_order_tom_experiment(actors, locs, g, length):
    poi = random.sample(actors, 3)
    loc_1 = random.sample(locs,1)
    loc_2 = random.sample(g[loc_1[0]], 1)
    loc_3 = random.sample([l for l in g[loc_2[0]] if l != loc_1[0]], 1)
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc_1, "path_type":"same"}
    event_dict[20] = {"name": "cross_paths","actors": poi[1:], "location": loc_2, "path_type":"same"}
    event_dict[21] = {"name":"move", "actor":poi[-1],"location":loc_3[0]}
    event_dict[22] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc_1[0], 'poi':poi}
    return event_dict, loc_2, experiment_info
    

def cross_path_overlap(actors, locs, g, mislead, length, n):
    poi = random.sample(actors, n)
    loc = random.sample(locs, 1)
    second_loc = random.choice(g[loc[0]])
    event_dict = {}
    event_dict[15] = {"name": "cross_paths","actors": poi, "location": loc, "path_type": "same"}
    event_dict[16] = {"name":"move", "actor":poi[-1], "location": second_loc}
    event_dict[17] = {"name": "exclusive_random", "actors": poi, "stop": 17 + mislead}
    event_dict[17 + mislead] = {"name": "mislead", "actors": poi}
    event_dict[17 + mislead+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, second_loc, experiment_info

def write_on_wall_then_erase(actors, locs, g, mislead, length):
    poi = random.sample(actors, 3)
    loc_1 = random.sample(locs,1)
    loc_2 = random.sample(g[loc_1[0]], 1)
    loc_3 = random.sample([l for l in g[loc_2[0]] if l != loc_1[0]], 1)
    event_dict = {}
    actions_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc_1, "path_type":"same"}
    actions_dict[10] = {'actor':poi[-1], 'action': 'draws a circle on the wall'}
    event_dict[20] = {"name": "cross_paths","actors": poi[1:], "location": loc_2, "path_type":"same", "prev": loc_1[0]}
    actions_dict[20] = {'actor':poi[-1], 'action': 'draws a circle on the wall'}
    event_dict[21] = {"name":"move", "actor":poi[-1],"location":loc_3[0]}
    event_dict[22] = {"name": "exclusive_random", "actors": poi, "stop": length}
    actions_dict[length-2] = {'actor':poi[-1], 'action': 'draws a circle on the wall'}
    experiment_info = {'cross path location': loc_1[0], 'poi':poi, "draw":[11, 22, length-2]}
    return event_dict, actions_dict, loc_2, experiment_info

def placed_objects(actors, locs, g, mislead, length):
    poi = random.sample(actors, 3)
    loc_1 = random.sample(locs,1)
    loc_2 = random.sample(g[loc_1[0]], 1)
    loc_3 = random.sample([l for l in g[loc_2[0]] if l != loc_1[0]], 1)
    event_dict = {}
    actions_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc_1, "path_type":"same"}
    actions_dict[10] = {'actor':poi[-1], 'action': 'draws a circle on the wall'}
    event_dict[20] = {"name": "cross_paths","actors": poi[1:], "location": loc_2, "path_type":"same", "prev": loc_1[0]}
    actions_dict[20] = {'actor':poi[-1], 'action': 'puts their phone on the ground'}
    event_dict[21] = {"name":"move", "actor":poi[-1],"location":loc_3[0]}
    event_dict[22] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc_1[0], 'poi':poi, "draw":[11, 22, length-2]}
    return event_dict, actions_dict, "No", experiment_info
    
def second_order_tom_extension(actors, locs, g, length, actions_done=False):
    poi = random.sample(actors, 3)
    loc_1 = random.sample(locs,1)
    loc_2 = random.sample(g[loc_1[0]], 1)
    loc_3 = random.sample([l for l in g[loc_2[0]] if l != loc_1[0]], 1)
    event_dict = {}
    actions_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc_1, "path_type":"same"}
    event_dict[20] = {"name": "cross_paths","actors": poi[1:], "location": loc_2, "path_type":"same", "prev": loc_1[0]}
    event_dict[21] = {"name":"move", "actor":poi[-1],"location":loc_3[0]}
    event_dict[22] = {"name": "exclusive_random", "actors": poi, "stop": length}
    
    shape = None
    if actions_done:
        shape = random.choice(['circle', 'square', 'triangle', 'oval', 'star'])
        actions_dict[10] = {'actor':poi[-1], 'action': f'draws a {shape} on the wall'}
        actions_dict[20] = {'actor':poi[-1], 'action': 'puts their phone on the ground'}

    experiment_info = {'cross path location': loc_1[0], 'poi':poi, "shape":shape}
    
    return event_dict, actions_dict, (loc_2[0], loc_3[0]), experiment_info

In [7]:
def find_k_unique_paths(graph, start, end, k):
    def dfs(node, path, visited, paths):
        if len(paths) >= k:  # Stop early if we found k paths
            return
        
        if node == end:  # If reached destination, store the path
            paths.append(list(path))
            return

        for neighbor in graph.get(node, []):  # Explore neighbors
            if neighbor not in visited:
                visited.add(neighbor)
                path.append(neighbor)

                dfs(neighbor, path, visited, paths)

                # Backtrack
                visited.remove(neighbor)
                path.pop()

    paths = []
    dfs(start, [start], {start}, paths)  # Start DFS
    return [len(p)-1 for p in paths[:k]],[p[1:] for p in paths[:k]]  # Return up to k paths

### Actual Experiment setup

In [8]:

df = pd.DataFrame({'Story':[], 'Tom_Label':[], 'P1':[], 'P2':[], 'WM_Label':[]})
possible_people = ["The shoe", "The ball", "The hat", "The apple", "The chess piece", "The bottle", "The flashlight", "The cup"]
human_names = ["Alice", "Bob", "Charlie", "Danny", "Edward", "Frank", "Georgia"]
num_people = 6

object_to_human = {possible_people[i]:human_names[i] for i in range(num_people)}

graph = { 
    "room_1": ["room_2", "the_hallway","room_5"],
    "room_2": ["room_1", "room_3","the_hallway"],
    "room_3": ["room_2", "room_4","the_hallway"],
    "room_4": ["room_3", "room_5","room_1"],
    "room_5": ["room_4", "room_1","room_2"],
    "the_hallway": ["room_1", "room_4","room_2"]
}

locations = list(graph.keys())
story_length = 30
num_trials = 100
mislead_distance = 3

random.seed(25)

for _ in range(num_trials):
    event_dict, action_dict, labels, experiment_dict = second_order_tom_extension(possible_people[:num_people], locations[:-1], graph, story_length, actions_done=False)
   

    sim = StorySimulator(
        people=possible_people[:num_people],
        locations=locations,
        relation="is_moved_to",
        params={'prompt': '3', 'type': 'cot'},
        graph=graph,
        events=event_dict,
        actions=action_dict
    )
    res = sim.run_simulation(story_length)
    
    story = sim.formal_to_story(res)
    
    #split_story.insert(story_length-1, f" {experiment_dict['poi'][1]} forgets what they last saw on the wall")
    # print(".\n".join(story))

    d = {'Story':[], 'Tom_Label':[], 'P1':[], 'P2':[], 'WM_Label':[]}
    d['P1'] = ",".join(experiment_dict['poi'][:-1]) if len(experiment_dict['poi'][:-1]) > 1 else experiment_dict['poi'][:-1]
    #print(d["P1"])
    d['P2'] = experiment_dict['poi'][-1]
    d['Story'].append(story)
    # d['Label'].append(movement[0])
    d['Tom_Label'].append(labels[0])
    d['WM_Label'].append(labels[0])
    df = pd.concat([df, pd.DataFrame(d)])

In [9]:
object_to_human

{'The shoe': 'Alice',
 'The ball': 'Bob',
 'The hat': 'Charlie',
 'The apple': 'Danny',
 'The chess piece': 'Edward',
 'The bottle': 'Frank'}

In [10]:
df.head()

,Story,Tom_Label,P1,P2,WM_Label
0,The apple is moved to room_4. The bottle is mo...,the_hallway,"The apple,The shoe",The ball,the_hallway
0,The hat is moved to room_4. The hat is moved t...,room_3,"The shoe,The apple",The chess piece,room_3
0,The apple is moved to room_4. The ball is move...,room_4,"The hat,The shoe",The chess piece,room_4
0,The hat is moved to room_1. The ball is moved ...,room_4,"The bottle,The hat",The chess piece,room_4
0,The shoe is moved to room_4. The ball is moved...,room_1,"The apple,The shoe",The hat,room_1


In [11]:
for i ,row in df.iterrows():
    print("\n".join(row['Story'].split(".")))
    print('---')

The apple is moved to room_4
 The bottle is moved to room_1
 The bottle is moved to room_5
 The hat is moved to room_2
 The shoe is moved to room_4
 The bottle is moved to room_1
 The shoe is moved to room_3
 The chess piece is moved to room_4
 The ball is moved to room_4
 The apple is moved to room_3
 The ball is moved to room_3
 The shoe is moved to the_hallway
 The apple is moved to the_hallway
 The apple is moved to room_2
 The bottle is moved to room_2
 The hat is moved to room_3
 The hat is moved to the_hallway
 The apple is moved to the_hallway
 The hat is moved to room_2
 The apple is moved to room_1
 The ball is moved to the_hallway
 The ball is moved to room_4
 The bottle is moved to room_1
 The chess piece is moved to room_1
 The chess piece is moved to the_hallway
 The chess piece is moved to room_2
 The bottle is moved to room_5
 The chess piece is moved to room_3
 The hat is moved to room_3
 The chess piece is moved to room_2
---
The hat is moved to room_4
 The hat is mov

In [12]:
fewshot_examples = []

### Run Model

In [13]:
intial_prompt = f"Read the following story and answer the question at the end. Note that all subjects start in {sim.locations[-1].replace('_',' ')}. Subjects of the story that are in the same location can see where eachother go when someone leaves. If subjects are in different locations, they cannot see eachother. There is only one of each subject in the story, so a subject being moved into one room implies that they are no longer in the other. Output your final answer as one word at the end of your response."

tom_responses, wm_responses_1, wm_responses_2 = [], [], []
fewshot = False
model_choice = "gpt-4"
tom_total, wm_total = 0, 0
for _ ,row in df.iterrows():
    p = row['P1'].split(',')
    formatted = []
    tom_answer = ''
    wm_answer = ''
    if fewshot:
        #formatted = [f'{ex[0]}\nQ: Where does {ex[1][0]} think {ex[1][1]} thinks {ex[1][2]} is?\nA: {ex[2]}' for ex in fewshot_examples]
        formatted = [f'{ex[0]}\nQ: Where does {ex[1][0]} think {ex[1][1]} is?\nA: {ex[2]}' for ex in fewshot_examples]
        formatted = '\n'.join(formatted)
        prompt = f"{intial_prompt}\n{formatted}\n{row['Story']}\nQ: Where does {p[0]} think {row['P2']} is?\nA:"
        #prompt = f"{intial_prompt}\n{formatted}\n{row['Story']}\nQ: Who has seen more drawings, {p[0]} or {row['P2']}?\nA:"
        print(prompt)
        print('*****')
        #answer = prompt_gpt(prompt, model_choice)
    else:
        prompt_prefix = f"{intial_prompt}\n{row['Story']}.\n"
        # Where does person 0 think the target is
        #mapped_prefix = ' '.join(list(map(lambda x: object_to_human[x] if x in object_to_human else x, prompt_prefix.split(' '))))
        #mapped_prefix = mapped_prefix.replace("is moved", "enters")
        mapped_prefix = prompt_prefix
        for x in object_to_human.items():
            mapped_prefix = mapped_prefix.replace(x[0], x[1])
            
        tom_prompt = f"{mapped_prefix.replace('is moved to', 'moves to')}Q: Where does {object_to_human[p[0]]} think {object_to_human[p[1]]} thinks {object_to_human[row['P2']]} is?\nA:"
        wm_prompt_1 = f"{mapped_prefix.replace('is moved to', 'moves to')}Q: When {object_to_human[p[0]]} and {object_to_human[p[1]]} were in the same room as {object_to_human[row['P2']]}, where did {object_to_human[row['P2']]} go?\nA:"
        wm_prompt_2 = f"{prompt_prefix}Q: When {p[0]} and {p[1]} were in the same room as {row['P2']}, where was {row['P2']} moved to?\nA:"
        tom_total += len(tom_prompt)
        wm_total += len(wm_prompt_1) + len(wm_prompt_2)
        tom_answer = prompt_gpt(tom_prompt, model_choice)
        wm_answer_1 = prompt_gpt(wm_prompt_1, model_choice)
        wm_answer_2 = prompt_gpt(wm_prompt_2, model_choice)
        print(f'{tom_prompt}\n\n{wm_prompt_1}\n\n{wm_prompt_2}')
        #print("=====")
        #answer = prompt_gpt(prompt, model_choice)
    tom_responses.append(tom_answer)
    wm_responses_1.append(wm_answer_1)
    wm_responses_2.append(wm_answer_2)        
df['TOM Responses'] = tom_responses
df['WM Responses Humans'] = wm_responses_1
df['WM Responses Objects'] = wm_responses_2

Read the following story and answer the question at the end. Note that all subjects start in the hallway. Subjects of the story that are in the same location can see where eachother go when someone leaves. If subjects are in different locations, they cannot see eachother. There is only one of each subject in the story, so a subject being moved into one room implies that they are no longer in the other. Output your final answer as one word at the end of your response.
Danny moves to room_4. Frank moves to room_1. Frank moves to room_5. Charlie moves to room_2. Alice moves to room_4. Frank moves to room_1. Alice moves to room_3. Edward moves to room_4. Bob moves to room_4. Danny moves to room_3. Bob moves to room_3. Alice moves to the_hallway. Danny moves to the_hallway. Danny moves to room_2. Frank moves to room_2. Charlie moves to room_3. Charlie moves to the_hallway. Danny moves to the_hallway. Charlie moves to room_2. Danny moves to room_1. Bob moves to the_hallway. Bob moves to room

In [ ]:
# Cost of input
(0.3 * (tom_total + wm_total) / 1000000) * 3

0.0748035

In [23]:
tom_total/100

1241.65

In [38]:
# Cost of output:
(0.3 * (sum(df['TOM Responses'].apply(lambda x: len(x))) + sum(df['WM Responses'].apply(lambda x: len(x)))) / 1000000) * 7

1.9724249999999999

In [ ]:
for trial in range(num_trials):
    print(df[' Responses'].iloc[trial], f", {df['Tom_Label'].iloc[trial]}")
    print('********=============************')

room_3 , the_hallway
********=============************
the_hallway , room_3
********=============************
room_5 , room_4
********=============************
room_4 , room_4
********=============************
room_1 , room_1
********=============************
Room_5 , room_2
********=============************
The story does not provide information on when The ball and The hat were in the same room as The chess piece. , the_hallway
********=============************
the_hallway , room_3
********=============************
the_hallway , room_1
********=============************
room_2 , room_2
********=============************
room_4 , the_hallway
********=============************
Room_3 , room_4
********=============************
room_2 , room_5
********=============************
room_5 , room_5
********=============************
room_2 , room_5
********=============************
room_2 , room_1
********=============************
room_2 , the_hallway
********=============************
the_hallway 

In [48]:
# print(df['TOM Responses'].iloc[0], f", {df['Tom_Label'].iloc[0][0]}")

In [49]:
# for i in range(10):
#     print(df['TOM Responses'][df['TOM Responses'].apply(lambda x: '**Answer:**' not in x or 'A:' not in x or 'Conclusion:')].iloc[i])
#     print('/////////////////////////////////////')

### Compute Accuracy

In [14]:
def compute_score_unsure(label, response):
    #label = label.split(',')[0][2:-1]
    base = label.split("_")[0] if not label.startswith(sim.locations[-1]) else label
    response = response.split("\n")[-1]
    if response.count(base) <= 1:
        return str(label in response or label.replace('_'," ") in response)
    elif 'Therefore,' in response:
        return str(label in response.split('Therefore,')[-1] or label in response.split('Therefore,')[-1])
    return f'{response}, {label}'

def compute_score_unsure_r1(label, response):
    label = label.split(',')[0][2:-1]
    # base = label.split("_")[0] if not label.startswith(sim.locations[-1]) else label
    if 'Answer:' in response:
        return str(label in response.split('Answer:')[-1] or label.replace('_',' ') in response.split('Answer:')[-1])
    elif 'A:' in response:
        return str(label in response.split('A:')[-1] or label.replace('_',' ') in response.split('A:')[-1])
    elif 'Conclusion:' in response:
        return str(label in response.split('Conclusion:')[-1] or label.replace('_',' ') in response.split('Conclusion:')[-1])
    else:
        return f'{response}, {label}'

outs= df.apply(lambda x: compute_score_unsure(x['Tom_Label'], x['TOM Responses']), axis=1)
known = [k for k in outs if k == 'True' or k == 'False']
unkown = [k for k in outs if k != 'True' and k != 'False']
print(sum([1 for k in known if k == 'True']))
for i in range(len(outs)):
    if outs.iloc[i] != 'True' and outs.iloc[i] != 'False':
        clean_out = outs.iloc[i].split('<think>')[-1]
        print(f'{clean_out}')
        print(f'Index {i}')
        print("\n-////==============////-\n")

38


In [19]:
outs= df.apply(lambda x: compute_score_unsure(x['WM_Label'], x['WM Responses Objects']), axis=1)
known = [k for k in outs if k == 'True' or k == 'False']
unkown = [k for k in outs if k != 'True' and k != 'False']
print(sum([1 for k in known if k == 'True']))
for i in range(len(outs)):
    if outs.iloc[i] != 'True' and outs.iloc[i] != 'False':
        clean_out = outs.iloc[i].split('<think>')[-1]
        print(f'{clean_out}')
        print(f'Index {i}')
        print("\n-////==============////-\n")

11


In [22]:
known[6]

'False'

In [32]:
sum([1 if '<think>' in i else 0 for i in unkown])

0

In [9]:
[print(i) if '<think>' in i else 0 for i in unkown]

<think>
Okay, let's try to figure out where Charlie thinks Alice thinks Danny is. Hmm, this seems a bit complicated, but I'll take it step by step.

First, I need to track the movements of all the characters, especially Alice, Danny, and Charlie. The key here is that characters can only see each other if they're in the same location. So, whenever someone moves, the people in the same location can see where they go. If they're in different places, they can't see each other's movements.

Let me start by listing all the events in order. Let's go through each action one by one and note where each character is at each step. Maybe a timeline would help.

1. Bob enters room_4. So Bob is in room_4. Others are in the hallway.
2. Bob enters room_3. So Bob moves from room_4 to room_3. At this point, anyone in room_4 (no one else) and the hallway (others) can't see him move except maybe if someone was in room_4 when he left. Wait, when Bob leaves room_4 to go to room_3, the people in room_4 (which

[None, None, None, None, None, None]

In [31]:
print(df['TOM Responses'].iloc[6])

room_4


In [15]:
#df.apply(lambda x: x['Label'] in x['Responses'], axis=1)
#df.to_csv("saved_results/gpt4_triplets_100.csv", index=False) 
#df.apply(lambda x: f"{x['TOM Responses']}, {x['WM Responses']}, {x['Tom_Label']}", axis=1).to_list()

In [61]:
df[df.apply(lambda x: x['Tom_Label'] not in x['TOM Responses'] and x['Tom_Label'] not in x['WM Responses'], axis=1)]

,Story,Tom_Label,P1,P2,WM_Label,TOM Responses,WM Responses
0,Danny enters room_4. Frank enters room_1. Alic...,the_hallway,"Danny,Alice",Bob,the_hallway,room_4,Bob went to the hallway.
0,Charlie enters room_4. Charlie enters room_5. ...,room_3,"Alice,Danny",Edward,room_3,Room 3,Edward entered the hallway.
0,Alice enters room_1. Danny enters room_4. Alic...,room_4,"Charlie,Alice",Edward,room_4,room_3,Edward entered room_3.
0,Bob enters room_4. Bob enters room_3. Alice en...,the_hallway,"Charlie,Alice",Danny,the_hallway,the hallway,Danny entered the hallway.
0,Edward enters room_1. Edward enters the_hallwa...,the_hallway,"Charlie,Alice",Danny,the_hallway,room_4,Danny entered room_4.
0,Danny enters room_1. Bob enters room_1. Alice ...,room_2,"Bob,Charlie",Danny,room_2,room_1,Danny went to room_5.
0,Edward enters room_2. Bob enters room_1. Edwar...,the_hallway,"Charlie,Alice",Frank,the_hallway,room_1,Frank entered the hallway.
0,Danny enters room_4. Alice enters room_4. Dann...,the_hallway,"Danny,Charlie",Edward,the_hallway,Room 3,Edward entered the hallway.
0,Charlie enters room_2. Edward enters room_4. E...,the_hallway,"Alice,Bob",Frank,the_hallway,room_1,Frank entered the hallway.
0,Edward enters room_1. Edward enters room_2. Al...,room_3,"Danny,Bob",Frank,room_3,room_4,Frank entered the hallway.


In [11]:
d = pd.read_csv('saved_results/deepseek_r1_second_order_tom_set_2.csv')
d.head()

,Story,Tom_Label,P1,P2,WM_Label,TOM Responses,WM Responses
0,Danny enters room_4. Frank enters room_1. Alic...,"('the_hallway', 'room_4')","Danny,Alice",Bob,room_3,"<think>\nOkay, let's try to figure out where B...","<think>\nOkay, let's try to figure out where D..."
1,Charlie enters room_4. Charlie enters room_5. ...,"('room_3', 'the_hallway')","Alice,Danny",Edward,room_4,"<think>\nOkay, let's try to figure out where E...","<think>\nOkay, let's try to figure out where A..."
2,Alice enters room_1. Danny enters room_4. Alic...,"('room_4', 'room_3')","Charlie,Alice",Edward,room_5,"<think>\nOkay, let's try to figure out where E...","<think>\nOkay, let's try to figure out where C..."
3,Bob enters room_2. Charlie enters room_1. Edwa...,"('room_4', 'room_3')","Frank,Charlie",Edward,room_5,"<think>\nOkay, let's try to figure out where E...","<think>\nOkay, let's try to figure out where F..."
4,Bob enters room_4. Danny enters room_4. Frank ...,"('room_4', 'room_1')","Bob,Frank",Charlie,room_3,"<think>\nOkay, let's try to figure out where C...","<think>\nOkay, let's try to figure out where B..."


In [ ]:
# print('\n'.join(d['Story'].to_list()[0].split('.')))
# print(d['Label'].to_list()[0])
d['Pred'] = d.apply(lambda x: compute_score_unsure_r1(x['Label'], x['Responses']), axis=1)

Bob jumps in hole_2
 Charlie jumps in hole_1
 Charlie jumps in hole_3
 Charlie jumps out to the field
 Bob jumps out to the field
 Bob jumps in hole_2
 Charlie jumps in hole_4
 Charlie jumps in hole_3
 Danny jumps in hole_2
 Bob jumps out to the field
 Alice jumps in hole_2
 Alice jumps in hole_3
 Alice jumps out to the field
 Bob jumps in hole_1
 Charlie jumps out to the field
 Bob jumps in hole_3
 Bob jumps out to the field
 Bob jumps in hole_2
 Bob jumps in hole_1
 Bob jumps out to the field
 Charlie jumps in hole_4
 Charlie jumps in hole_1
 Charlie jumps out to the field
 Bob jumps in hole_4
 Charlie jumps in hole_1
hole_3


In [149]:
wrong = d[d['Pred'] == 'False']
print('\n'.join(wrong['Story'].to_list()[10].split('.')))
print(wrong['Label'].to_list()[10])
print(wrong['P1'].to_list()[10])
print(wrong['P2'].to_list()[10])
print(wrong['Responses'].to_list()[10])



Danny jumps in hole_1
 Danny jumps in hole_2
 Danny jumps in hole_3
 Danny jumps in hole_2
 Bob jumps in hole_1
 Alice jumps in hole_2
 Alice jumps out to the field
 Alice jumps in hole_1
 Bob jumps in hole_3
 Charlie jumps in hole_1
 Charlie jumps in hole_3
 Charlie jumps in hole_4
 Charlie jumps in hole_1
 Danny jumps in hole_3
 Danny jumps out to the field
 Danny jumps in hole_2
 Danny jumps in hole_1
 Danny jumps in hole_3
 Danny jumps in hole_2
 Alice jumps out to the field
 Alice jumps in hole_2
 Danny jumps out to the field
 Danny jumps in hole_4
 Danny jumps in hole_3
 Danny jumps in hole_2
hole_4
Bob
Charlie
Bob thinks Charlie is in hole_1.
